In [1]:
import torch
import torch.nn as nn
torch.cuda.is_available()
import pandas as pd

In [2]:
from isaac.dataset import read_dataset, prepare_dataset
from isaac.constants import POSITION_COLS, MASS_CLASS_COLS, BASIC_TRAINING_COLS, FORCE_CLASS_COLS, PUCK_SQUARE_DISTANCES, PUCK_ANGLE_FEATURES
from isaac.sanity import class_proportions
from isaac.models import initialise_model
from isaac.training import training_loop
import matplotlib.pyplot as plt
import numpy as np
import matplotlib2tikz

In [3]:
NORMALISE_DATA = True
BATCH_SIZE = 128
EPOCHS = 25
SEQ_END = 1800

In [4]:
TR_COLS = BASIC_TRAINING_COLS

In [5]:
train_trials = read_dataset("data/train_passive_trials.h5", n_trials=3500, cols=TR_COLS)
val_trials = read_dataset("data/val_passive_trials.h5", n_trials=500, cols=TR_COLS)

100%|██████████| 500/500 [00:04<00:00, 120.67it/s]


# MASS TRAINING

In [6]:
INPUT_DIM = len(TR_COLS)    # input dimension
HIDDEN_DIM = 25  # hidden layer dimension
N_LAYERS = 4     # number of hidden layers
OUTPUT_DIM = 3   # output dimension
DROPOUT = 0.5
network_params = (INPUT_DIM, HIDDEN_DIM, N_LAYERS, OUTPUT_DIM, DROPOUT)

In [7]:
labels = [1, 2, 3, 4]
stats_dfs = []

loaders, scaler = prepare_dataset([train_trials, val_trials], class_columns=MASS_CLASS_COLS, 
                                  training_columns=TR_COLS, batch_size=BATCH_SIZE, 
                                  normalise_data=NORMALISE_DATA)

for step, label in zip([1, 2, 3, 4], labels):    
    for seed in [0, 42, 72]:
        df = pd.DataFrame(columns=["seq_resolution", "Epoch", "Loss"])


        model, error, optimizer = initialise_model(network_params, lr=0.01, seed=seed)
        epoch_losses, epoch_accuracies, best_model = training_loop(model, optimizer, error, loaders[0], 
                                                                   loaders[1], EPOCHS, seq_end=SEQ_END, 
                                                                   step_size=step)
                
        df["Epoch"] = np.arange(EPOCHS)
        df["Loss"] = epoch_losses
        df["Train Accuracy"] = epoch_accuracies[0]
        df["Val Accuracy"] = epoch_accuracies[1]
        df["seq_resolution"] = str(label)
        stats_dfs.append(df)
        
stats = pd.concat(stats_dfs)
stats.to_hdf("sequence_resolution_plots/mass_stats.h5", key="stats")

100%|██████████| 500/500 [00:00<00:00, 1020.72it/s]
Train_loss (0.90)	 Train_acc (58.17)	 Val_acc (51.40): 100%|██████████| 25/25 [08:38<00:00, 20.68s/it]
Train_loss (0.67)	 Train_acc (71.77)	 Val_acc (54.60): 100%|██████████| 25/25 [08:37<00:00, 20.69s/it]
Train_loss (0.82)	 Train_acc (62.34)	 Val_acc (56.00): 100%|██████████| 25/25 [08:45<00:00, 21.05s/it]
Train_loss (0.65)	 Train_acc (68.06)	 Val_acc (50.80): 100%|██████████| 25/25 [04:30<00:00, 10.79s/it]
Train_loss (0.75)	 Train_acc (67.89)	 Val_acc (48.60): 100%|██████████| 25/25 [04:30<00:00, 10.84s/it]
Train_loss (0.71)	 Train_acc (70.31)	 Val_acc (54.60): 100%|██████████| 25/25 [04:28<00:00, 10.73s/it]
Train_loss (0.60)	 Train_acc (71.89)	 Val_acc (49.00): 100%|██████████| 25/25 [03:01<00:00,  7.26s/it]
Train_loss (0.86)	 Train_acc (63.49)	 Val_acc (46.20): 100%|██████████| 25/25 [03:02<00:00,  7.30s/it]
Train_loss (0.67)	 Train_acc (73.46)	 Val_acc (55.80): 100%|██████████| 25/25 [03:01<00:00,  7.29s/it]
Train_loss (0.62)	 Tr

# FORCE TRAINING

In [8]:
labels = [1, 2, 3, 4]
stats_dfs = []

loaders, scaler = prepare_dataset([train_trials, val_trials], class_columns=FORCE_CLASS_COLS, 
                                  training_columns=TR_COLS, batch_size=BATCH_SIZE, 
                                  normalise_data=NORMALISE_DATA)

for step, label in zip([1, 2, 3, 4], labels):    
    for seed in [0, 42, 72]:
        df = pd.DataFrame(columns=["seq_resolution", "Epoch", "Loss"])


        model, error, optimizer = initialise_model(network_params, lr=0.01, seed=seed)
        epoch_losses, epoch_accuracies, best_model = training_loop(model, optimizer, error, loaders[0], 
                                                                   loaders[1], EPOCHS, seq_end=SEQ_END, 
                                                                   step_size=step)
                
        df["Epoch"] = np.arange(EPOCHS)
        df["Loss"] = epoch_losses
        df["Train Accuracy"] = epoch_accuracies[0]
        df["Val Accuracy"] = epoch_accuracies[1]
        df["seq_resolution"] = str(label)
        stats_dfs.append(df)
        
stats = pd.concat(stats_dfs)
stats.to_hdf("sequence_resolution_plots/force_stats.h5", key="stats")

100%|██████████| 500/500 [00:00<00:00, 918.48it/s]
Train_loss (0.67)	 Train_acc (67.26)	 Val_acc (55.60): 100%|██████████| 25/25 [08:55<00:00, 21.30s/it]
Train_loss (0.68)	 Train_acc (57.03)	 Val_acc (48.20): 100%|██████████| 25/25 [08:58<00:00, 21.61s/it]
Train_loss (0.62)	 Train_acc (67.00)	 Val_acc (56.00): 100%|██████████| 25/25 [08:55<00:00, 21.34s/it]
Train_loss (0.64)	 Train_acc (64.51)	 Val_acc (51.20): 100%|██████████| 25/25 [04:35<00:00, 11.07s/it]
Train_loss (0.63)	 Train_acc (72.54)	 Val_acc (53.20): 100%|██████████| 25/25 [04:36<00:00, 11.10s/it]
Train_loss (0.67)	 Train_acc (61.54)	 Val_acc (51.20): 100%|██████████| 25/25 [04:35<00:00, 11.02s/it]
Train_loss (0.57)	 Train_acc (70.34)	 Val_acc (62.80): 100%|██████████| 25/25 [03:01<00:00,  7.25s/it]
Train_loss (0.59)	 Train_acc (74.71)	 Val_acc (63.60): 100%|██████████| 25/25 [03:01<00:00,  7.29s/it]
Train_loss (0.63)	 Train_acc (68.31)	 Val_acc (55.60): 100%|██████████| 25/25 [03:01<00:00,  7.31s/it]
Train_loss (0.62)	 Tra